In [1]:
import os
from PIL import Image, ImageEnhance
import numpy as np

In [2]:
!pip install -q kaggle

In [ ]:
# Download the dataset
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign -p /work/flemingc/nvan21/projects/COMS_573_Project/Data

In [ ]:
!unzip /work/flemingc/nvan21/projects/COMS_573_Project/Data/gtsrb-german-traffic-sign.zip -d /work/flemingc/nvan21/projects/COMS_573_Project/Data

In [7]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [8]:
from PIL import Image, ImageDraw

In [9]:
def add_black_squares(image, num_squares=7, square_size_range=(15, 70),):
    """
    Adds black squares to the image to occlude parts of it, and resizes the image to a target size.
    
    Parameters:
    - image: PIL Image object
    - num_squares: Number of squares to add to the image
    - square_size_range: Tuple (min_size, max_size) for random square sizes
    - target_size: Target size for the image (width, height)
    
    Returns:
    - Augmented PIL Image with black squares resized to the target size
    """

    
    width, height = image.size
    size_ref=min(width, height)
    small_square= int(size_ref/12)
    big_square=int(size_ref/6)
    if small_square ==0:
        small_square+=1
    if small_square == big_square:
        big_square+=1

    square_size_range=(small_square, big_square)
    draw = ImageDraw.Draw(image)

    for _ in range(num_squares):
        # Randomly determine the size of the square
        square_size = random.randint(square_size_range[0], square_size_range[1])

        # Randomly determine the top-left corner of the square
        top_left_x = random.randint(0, width - square_size)
        top_left_y = random.randint(0, height - square_size)

        # Define the bottom-right corner of the square
        bottom_right_x = top_left_x + square_size
        bottom_right_y = top_left_y + square_size

        # Draw a black square
        draw.rectangle([top_left_x, top_left_y, bottom_right_x, bottom_right_y], fill="black")

    return image

In [10]:
def augment_images(input_folder, output_folder, num_squares=7, square_size_range=(15, 70)):
    """
    Augments all images in the input folder by adding black squares and saves them to the output folder.
    
    Loops through subfolders in input_folder (representing class directories) and augments images in each class.
    
    Parameters:
    - input_folder: Path to the folder containing images to augment (with subfolders for each class)
    - output_folder: Path to the folder where augmented images will be saved
    - num_squares: Number of black squares to add to each image
    - square_size_range: Range for the size of the squares
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all subfolders (each representing a class)
    for class_folder in os.listdir(input_folder):
        class_path = os.path.join(input_folder, class_folder)

        # Check if it's a directory (class folder)
        if os.path.isdir(class_path):
            # Create an output folder for the class if it doesn't exist
            class_output_folder = os.path.join(output_folder, class_folder)
            if not os.path.exists(class_output_folder):
                os.makedirs(class_output_folder)
            
            # Loop through all images in the class folder
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)

                # Check if it's an image file
                if img_name.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff')):
                    # Open the image
                    image = Image.open(img_path)

                    # Apply black squares augmentation
                    augmented_image = add_black_squares(image.copy(), num_squares, square_size_range)

                    # Save the augmented image to the appropriate class folder in the output folder
                    augmented_img_path = os.path.join(class_output_folder, img_name)
                    augmented_image.save(augmented_img_path)
                    print(f"Saved augmented image: {augmented_img_path}")

In [ ]:
input_folder = '/work/flemingc/nvan21/projects/COMS_573_Project/Data/train'
output_folder = '/work/flemingc/nvan21/projects/COMS_573_Project/Data/train_augment'

augment_images(input_folder, output_folder)

In [ ]:
folder_paths = [
    "/work/flemingc/nvan21/projects/COMS_573_Project/Data/train",
    "/work/flemingc/nvan21/projects/COMS_573_Project/Data/train_augment"
]

In [ ]:
# Gather all subfolder paths (each folder can contain multiple subfolders for different classes)
subfolder_paths = []
for folder_path in folder_paths:
    subfolder_paths.append([os.path.join(folder_path, subfolder) for subfolder in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, subfolder))])

# Randomly sample 10 images (this will sample the same 10 images from all subfolders in each folder)
# Select a random subfolder from the first folder to get image names
image_names = os.listdir(subfolder_paths[0][0])  # Get image names from the first subfolder of the first folder

# Randomly sample 10 images
sampled_images = random.sample(image_names, 10)

# Create a plot to display the images side by side
fig, axes = plt.subplots(10, len(folder_paths), figsize=(15, 30))  # 10 rows, number of columns equals the number of folder_paths

# Loop through the sampled images and display them from each folder and subfolder
for i, img_name in enumerate(sampled_images):
    for j, folder_path in enumerate(folder_paths):
        ax = axes[i, j]  # Select the subplot for this image/folder combination

        # Iterate through each subfolder within the folder
        for subfolder_path in subfolder_paths[j]:
            img_path = os.path.join(subfolder_path, img_name)
            if os.path.exists(img_path):
                img = mpimg.imread(img_path)
                ax.imshow(img)
                ax.axis('off')  # Hide axes for better image viewing
                if j == 0:
                    ax.set_ylabel(f"Image {i+1}")  # Label the first column with image number
                if i == 0:
                    ax.set_title(f"Folder {j+1}")  # Title the top row with folder number

# Display the plot
plt.tight_layout()
plt.show()